In [ ]:
import json
import os
import io
import base64

import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
# Load data function
def load_json_data(filename):
    filepath = os.path.join('..', 'data', filename)
    with open(filepath, 'r', encoding="utf-8") as file:
        return json.load(file)

# Global variables to store data
rooms_data = load_json_data('test_rooms.json')
exits_data = load_json_data('test_exits.json')

In [ ]:
# Create graph function
def create_graph():
    G = nx.MultiDiGraph()
    
    for room in rooms_data['rooms']:
        G.add_node(room['RoomID'], title=room['Title'])
    
    exit_map = {exit['ExitID']: exit for exit in exits_data['exits']}
    
    for room in rooms_data['rooms']:
        for exit_id in room.get('ExitID', []):
            exit = exit_map.get(exit_id)
            if exit:
                G.add_edge(
                    room['RoomID'],
                    exit['TargetRoom'],
                    key=exit_id,
                    direction=exit['Direction']
                )
    
    return G

In [ ]:
# Visualization function
def visualize_graph(G):
    fig, ax = plt.subplots(figsize=(14, 10))
    pos = nx.spring_layout(G, k=1.5, iterations=200)
    
    nx.draw_networkx_nodes(G, pos, node_color='lightblue', node_size=700, ax=ax)
    node_labels = {node: G.nodes[node]['title'] for node in G.nodes()}
    nx.draw_networkx_labels(G, pos, labels=node_labels, font_size=10, ax=ax)
    
    edge_list = list(G.edges(keys=True, data=True))
    edge_counts = {}
    
    for u, v, key, data in edge_list:
        edge_counts[(u, v)] = edge_counts.get((u, v), 0) + 1
    
    for (u, v), count in edge_counts.items():
        edges = [(u, v2, k, d) for (u, v2, k, d) in edge_list if v2 == v]
        num_edges = len(edges)
        rad_list = [0.0] if num_edges == 1 else np.linspace(-0.5, 0.5, num_edges)
        
        for (u, v, key, data), rad in zip(edges, rad_list):
            nx.draw_networkx_edges(
                G, pos,
                edgelist=[(u, v, key)],
                connectionstyle=f'arc3,rad={rad}',
                arrowsize=15,
                edge_color='gray',
                ax=ax
            )
            
            x1, y1 = pos[u]
            x2, y2 = pos[v]
            xm, ym = (x1 + x2) / 2, (y1 + y2) / 2
            
            dx, dy = x2 - x1, y2 - y1
            angle = np.arctan2(dy, dx)
            
            horizontal_scale = 0.05
            vertical_offset = rad * 0.1
            
            label_x = xm - horizontal_scale * np.sin(angle)
            label_y = ym + vertical_offset * np.cos(angle)
            
            label = data.get('direction', 'No Label')
            ax.text(
                label_x,
                label_y,
                label,
                fontsize=8,
                ha='center',
                va='center',
                bbox=dict(facecolor='white', edgecolor='none', pad=1)
            )
    
    ax.set_title("Room Layout Graph")
    ax.axis('off')
    
    # Convert plot to image
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    img_str = base64.b64encode(buf.read()).decode('utf-8')
    plt.close(fig)
    
    return img_str


In [ ]:
# Main application layout
def room_editor_app():
    # Create graph
    G = create_graph()
    
    # Create output widget for displaying messages
    output = widgets.Output()
    
    # Create image widget for displaying the graph
    image = widgets.Image()
    
    # Create button to update graph
    update_button = widgets.Button(description="Update Graph")
    
    # Update graph function
    def update_graph(b):
        with output:
            clear_output()
        img_str = visualize_graph(G)
        image.value = base64.b64decode(img_str)
    
    update_button.on_click(update_graph)
    
    # Layout
    app_layout = widgets.VBox([
        widgets.HTML("<h1>Room Editor</h1>"),
        update_button,
        image,
        output
    ])
    
    # Initial graph update
    update_graph(None)
    
    return app_layout

In [ ]:
# Display the app
display(room_editor_app())